Трекинг всех классов:

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import os
import threading
import cv2
from ultralytics import YOLO



# ----------- Константы -------------
CLASS_NAMES = ['white_helmet', 'other_helmet', 'safety_signs']
CLASS_COLORS = {
    0: (255, 255, 255),
    1: (0, 165, 255),
    2: (0, 0, 255),
}



# ----------------------------------------Выбор видео-----------------------------------
def select_video():
    path = filedialog.askopenfilename(title="Выберите видео", filetypes=[("MP4 файлы", "*.mp4"), ("Все файлы", "*.*")])
    if path:
        video_path_var.set(path)



# ------------------------------------Выбор папки для кадров-----------------------------------------------
def select_folder():
    path = filedialog.askdirectory(title="Выберите папку для кадров с нарушениями")
    if path:
        folder_path_var.set(path)



# -------------------------------Основная обработка YOLO------------------------------------
def run_detection(video_path, output_folder, allowed_other_helmet_val, allowed_white_helmet_val, imgsz_val, tracker_val):
    try:
        model = YOLO('runs/detect/train/weights/best.pt')
        model.fuse()

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise Exception("Ошибка при открытии видеофайла.")

        # Предварительное открытие окна, чтобы пользователь увидел его до начала анализа
        cv2.namedWindow("Детекция нарушений", cv2.WINDOW_NORMAL)
        dummy_frame_displayed = False

        while True:
            ret, preview_frame = cap.read()
            if not ret:
                raise Exception("Ошибка при предварительном чтении видео.")
            resized = cv2.resize(preview_frame, (0, 0), fx=0.5, fy=0.5)
            cv2.putText(resized, "Start detection", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.imshow("Детекция нарушений", resized)
            cv2.waitKey(1000)
            dummy_frame_displayed = True
            break

        if not dummy_frame_displayed:
            raise Exception("Невозможно отобразить окно видео.")

        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # вернуться к началу видео
        last_logged_time = -1  # Последнее сохранённое нарушение (секунда)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model.track(frame, iou=0.4, conf=0.5, persist=True, imgsz=imgsz_val, verbose=False, tracker=tracker_val)

            # процесс детекции
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
                ids = results[0].boxes.id.cpu().numpy().astype(int)
                classes = results[0].boxes.cls.cpu().numpy().astype(int)

                other_helmet_count = 0
                white_helmet_count = 0

                for box, id, class_id in zip(boxes, ids, classes):
                    label = CLASS_NAMES[class_id]
                    color = CLASS_COLORS.get(class_id, (0, 255, 0))
                    
                    if label == "other_helmet":
                        other_helmet_count += 1
                    elif label == "white_helmet":
                        white_helmet_count+=1

                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color, 2)
                    cv2.putText(
                        frame,
                        f"{label} ID {id}",
                        (box[0], box[1] - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        color,
                        2,
                    )

                # Отлов нарушений:
                current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
                current_time_int = int(current_time)
                minutes = current_time_int // 60
                seconds = current_time_int % 60

                if current_time_int != last_logged_time:
                    if other_helmet_count != allowed_other_helmet_val:
                        filename = f"!!!Нарушение!!!, несоответствующее кол-во СИНИХ касок на {minutes} мин {seconds} сек.jpg"
                        filepath = os.path.join(output_folder, filename)
                        cv2.imwrite(filepath, frame)
                        print(f"Нарушение сохранено: {filepath}")

                    if white_helmet_count != allowed_white_helmet_val:
                        filename = f"!!!Нарушение!!!, несоответствующее кол-во БЕЛЫХ касок на {minutes} мин {seconds} сек.jpg"
                        filepath = os.path.join(output_folder, filename)
                        cv2.imwrite(filepath, frame)
                        print(f"Нарушение сохранено: {filepath}")
                        last_logged_time = current_time_int
                    
                    if other_helmet_count > allowed_other_helmet_val or white_helmet_count > allowed_white_helmet_val:
                        last_logged_time = current_time_int

            # подсказкой пользователю
            cv2.putText(
                frame,
                "Press 'Q' to exit",
                (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 255),
                2,
            )

            resized_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
            cv2.imshow("Детекция нарушений", resized_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo("Готово", "Обработка завершена.")
    except Exception as e:
        messagebox.showerror("Ошибка", str(e))

# ----------- Старт обработки в потоке -------------
def start_detection():
    video_path = video_path_var.get()
    output_folder = folder_path_var.get()
    tracker = tracker_var.get()

    try:
        allowed_other = int(allowed_other_var.get())
        allowed_white = int(allowed_white_var.get())
        imgsz = int(imgsz_var.get())
    except ValueError:
        messagebox.showerror("Ошибка", "Введите корректные числовые параметры")
        return
    
    if not video_path or not output_folder:
        messagebox.showerror("Ошибка", "Укажите видео и папку для сохранения.")
        return

    threading.Thread(target=run_detection, args=(video_path, output_folder, allowed_other, allowed_white, imgsz, tracker), daemon=True).start()

# ----------- GUI интерфейс -------------
root = tk.Tk()
root.title("Детекция нарушений (YOLO)")
root.geometry("550x500")

video_path_var = tk.StringVar()
folder_path_var = tk.StringVar()
allowed_other_var = tk.StringVar(value="1")
allowed_white_var = tk.StringVar(value="1")
imgsz_var = tk.StringVar(value="640")
tracker_var = tk.StringVar(value="botsort.yaml")

# ---------------------------------------ВИДЕО-----------------------------------------------------
tk.Label(root, text="Выберите видео для анализа:").pack(pady=5)
tk.Entry(root, textvariable=video_path_var, width=60).pack()
tk.Button(root, text="Обзор видео", command=select_video).pack(pady=2)


# ---------------------------------------ПАПКА-----------------------------------------------------
tk.Label(root, text="Выберите папку для кадров с нарушениями:").pack(pady=10)
tk.Entry(root, textvariable=folder_path_var, width=60).pack()
tk.Button(root, text="Обзор папки", command=select_folder).pack(pady=2)


# --------------------------------------------------ВЫБОР СИНИХ КАСОК-------------------------------------------------
tk.Label(root, text="Максимальное допустимое кол-во синих касок:").pack(pady=10)
tk.Entry(root, textvariable=allowed_other_var, width=10).pack()


# --------------------------------------------------ВЫБОР БЕЛЫХ КАСОК-------------------------------------------------
tk.Label(root, text="Максимальное допустимое кол-во белых касок:").pack(pady=10)
tk.Entry(root, textvariable=allowed_white_var, width=10).pack()

# -------------------------------------------ВЫБОР ВХОДНОГО ИЗОБРАЖЕНИЯ-----------------------------------------------
tk.Label(root, text="Размер входного изиображения (imgsz):").pack(pady=10)
imgsz_combo = ttk.Combobox(root, textvariable=imgsz_var, values=["320", "640"], state="readonly", width=10)
imgsz_combo.pack()


# -----------------------------------------ВЫБОР ТРЕКЕРА---------------------------------------------
tk.Label(root, text="Выберите трекер:").pack(pady=10)
tracker_combo = ttk.Combobox(root, textvariable=tracker_var, values=["botsort.yaml", "bytetrack.yaml"], state="readonly", width=20)
tracker_combo.pack()


# ---------------------------------------КНОПКА ЗАПУСКА-----------------------------------------------------
tk.Button(root, text="Запустить детекцию", command=start_detection, height=2).pack(pady=20)


root.mainloop()

Model summary (fused): 92 layers, 25,841,497 parameters, 0 gradients, 78.7 GFLOPs
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во СИНИХ касок на 0 мин 0 сек.jpg
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во БЕЛЫХ касок на 0 мин 0 сек.jpg
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во СИНИХ касок на 0 мин 1 сек.jpg
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во БЕЛЫХ касок на 0 мин 1 сек.jpg
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во СИНИХ касок на 0 мин 2 сек.jpg
Нарушение сохранено: C:/Users/Honor/Downloads/new/папка_для_нарушений\!!!Нарушение!!!, несоответствующее кол-во БЕЛЫХ касок на 0 мин 2 сек.jpg
Model summary (fused): 92 layers, 25,841,497 parameters, 0 g